In [ ]:
import os
import requests
from supabase import create_client, Client
from dotenv import load_dotenv
import time

# --- 1. SETUP ---
load_dotenv()

# Connect to Supabase
SUPABASE_URL = os.environ.get("SUPABASE_URL")
SUPABASE_KEY = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

# Bright Data Config
BRIGHTDATA_ENDPOINT = os.environ.get("BRIGHTDATA_ENDPOINT")
BRIGHTDATA_HEADERS = {
    'Authorization': f'Bearer {os.environ.get("BRIGHTDATA_API_KEY")}'
}

# --- 2. BRIGHT DATA API CALL ---
def get_jobs_from_brightdata(role, city):
    """
    Calls the Bright Data scraper API for a specific role and city.
    """
    print(f"--- Fetching jobs for {role} in {city} ---")
    
    # This payload format depends on your specific Bright Data scraper
    payload = {
        'keyword': role,
        'location': city,
        "country":"US",
        "time_range":"Past week"
    }
    
    try:
        # Using a POST request as is common for scraper IDEs
        response = requests.post(
            BRIGHTDATA_ENDPOINT,
            json=payload,
            headers=BRIGHTDATA_HEADERS
        )
        response.raise_for_status() # Raises an error for bad responses (4xx or 5xx)
        
        # The structure of response.json() will depend on YOUR scraper's output
        # For this example, we assume it returns a list of job objects
        jobs = response.json()
        print(f"Found {len(jobs)} jobs.")
        return jobs
        
    except requests.exceptions.RequestException as e:
        print(f"Error calling Bright Data: {e}")
        return []

# --- 3. SUPABASE DATA HANDLING ---
def get_or_create_company(company_name, company_website=None, logo_url=None):
    """
    Intelligently gets a company ID. If the company doesn't exist, it creates it.
    This is an "upsert" (update or insert).
    """
    
    # .upsert() is perfect for this.
    # It will find a company with that name, or create a new one.
    data, count = supabase.table('companies').upsert(
        {
            'name': company_name,
            'website_url': company_website,
            'logo_url': logo_url
        },
        on_conflict='name' # This tells Supabase to match based on the 'name' column
    ).execute()
    
    # Return the ID of the company (whether new or existing)
    return data[0]['id']

def insert_job(job, company_id):
    """
    Inserts a single job into the 'jobs' table,
    first checking if it already exists based on the 'apply_url'.
    """
    try:
        # Check if job already exists (based on our unique constraint)
        existing = supabase.table('jobs').select('id').eq('apply_url', job['apply_url']).execute()
        
        if existing.data:
            print(f"Skipping duplicate: {job['title']}")
            return

        # Prepare the job record for insertion
        job_record = {
            'title': job['title'],
            'description': job['description'],
            'company_id': company_id,
            'location_name': job.get('location', 'N/A'), # Use .get() for safety
            'location_type': job.get('location_type', 'On-Site'),
            'apply_url': job['apply_url'],
            'posted_at': job.get('posted_at_timestamp', 'now()'),
            'source': 'BrightData',
            'is_paid': False,
            'is_featured': False
        }
        
        # Insert the new job
        supabase.table('jobs').insert(job_record).execute()
        print(f"Successfully inserted: {job['title']}")

    except Exception as e:
        print(f"Error inserting job {job.get('title')}: {e}")

# --- 4. MAIN SCRIPT LOGIC ---
def main():
    """
    Main function to run the backfill process.
    """
    
    # START SMALL (as you suggested)
    # This list will grow to 100s of combinations
    targets = [
        # California (Bay Area)
        ("AI Engineer", "San Francisco, CA"),
        ("AI Engineer", "San Jose, CA"),
        ("AI Engineer", "Palo Alto, CA"),
        ("AI Engineer", "Mountain View, CA"),
        ("AI Engineer", "Oakland, CA"),
        ("AI Engineer", "Santa Clara, CA"),
        ("AI Engineer", "Sunnyvale, CA"),
        # California (SoCal)
        ("AI Engineer", "Los Angeles, CA"),
        ("AI Engineer", "San Diego, CA"),
        # Washington
        ("AI Engineer", "Seattle, WA"),
        ("AI Engineer", "Bellevue, WA"),
        ("AI Engineer", "Redmond, WA"),
        # Massachusetts
        ("AI Engineer", "Boston, MA"),
        ("AI Engineer", "Cambridge, MA"),
        ("AI Engineer","Somerville, MA"),
        # DC Metro
        ("AI Engineer", "Washington, D.C."),
        ("AI Engineer", "Arlington, VA"),
        # Maryland
        ("AI Engineer", "Baltimore, MD"),
        ("AI Engineer", "Rockville, MD"),
        ("AI Engineer", "Bethesda, MD"),
        ("AI Engineer", "Gaithersburg, MD"),
        ("AI Engineer", "Columbia, MD"),
        ("AI Engineer", "Silver Spring, MD"),
    ]
    
    for role, city in targets:
        # 1. Get data from Bright Data
        # IMPORTANT: Bright Data's response format must be mapped here.
        # We assume `jobs` is a list of dicts like:
        # { 'title': '...', 'company_name': '...', 'description': '...', 'apply_url': '...', ... }
        jobs = get_jobs_from_brightdata(role, city)
        
        if not jobs:
            continue
            
        for job in jobs:
            # 2. Get or create the company
            # You must map your Bright Data fields to these
            company_id = get_or_create_company(
                job['company_name'],
                job.get('company_website'),
                job.get('company_logo')
            )
            
            # 3. Insert the job
            insert_job(job, company_id)
            
            # Rate limit ourselves to be kind to Supabase
            time.sleep(0.5) 
            
    print("--- Backfill complete ---")

if __name__ == "__main__":
    main()

In [37]:
import os
import requests
import pandas as pd
from supabase import create_client, Client
from dotenv import load_dotenv
import time
import json  # <-- 1. IMPORT THE JSON LIBRARY

# --- 1. SETUP ---
# Make sure you have:
# pip install pandas supabase-client python-dotenv requests

load_dotenv()

# --- Supabase Credentials ---
SUPABASE_URL = os.environ.get("SUPABASE_URL")
SUPABASE_KEY = os.environ.get("SUPABASE_KEY")
if not SUPABASE_URL or not SUPABASE_KEY:
    raise EnvironmentError("Supabase credentials not found in .env file")
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

# --- Bright Data Credentials ---
BRIGHTDATA_API_KEY = os.environ.get("BRIGHTDATA_API_KEY")
if not BRIGHTDATA_API_KEY:
    raise EnvironmentError("Bright Data API key not found in .env file")

# --- FILE & API CONFIG ---
LOG_FILE = 'api_logs_2025-11-16_14-22-09.csv' # The log file you uploaded

# This endpoint is not used, but left here from your script
DOWNLOAD_ENDPOINT = "https://api.brightdata.com/datasets/v3/snapshot/"

BRIGHTDATA_HEADERS = {
    'Authorization': f'Bearer {BRIGHTDATA_API_KEY}'
}

# --- 2. SUPABASE HELPER FUNCTIONS ---
def get_or_create_company(company_name, company_website=None, logo_url=None):
    clean_name = str(company_name).strip()
    if not clean_name:
        print("Skipping company: company_name is empty.")
        return None

    try:
        company_data = {'name': clean_name}
        if company_website: company_data['website_url'] = company_website
        if logo_url: company_data['logo_url'] = logo_url

        print(company_data)

        # FIX: capture proper response object
        response = supabase.table('companies').upsert(
            [company_data],
            on_conflict='name'
        ).execute()

        rows = response.data   # <-- This is always a list

        if rows and 'id' in rows[0]:
            return rows[0]['id']

        # fallback if no ID returned
        response = supabase.table('companies').select('id').eq('name', clean_name).execute()

        rows = response.data
        if rows and 'id' in rows[0]:
            return rows[0]['id']

        print(f"❌ Error: Could not get or create company ID for {clean_name}")
        return None

    except Exception as e:
        print(f"❌ Error upserting company {clean_name}: {e}")
        return None

def insert_job(job_row, company_id):
    """
    Upserts a single job (as a dict) into the 'jobs' table.
    Uses 'url' as the unique key.
    """
    title = job_row.get('job_title', 'No Title Provided')
    
    try:
        apply_url = job_row.get('apply_link') or job_row.get('url')  
        
        if not apply_url:
            print(f"Skipping job ({title}): 'url' field is missing.")
            return

        # --- Parse Location Type ---
        parsed_location_type = 'On-Site' # Default
        title_lower = title.lower()
        summary_lower = job_row.get('job_summary', '').lower()
        job_location_lower = job_row.get('job_location', '').lower()

        if 'remote' in title_lower or 'remote' in summary_lower or 'remote' in job_location_lower:
            parsed_location_type = 'Remote'
        elif 'hybrid' in title_lower or 'hybrid' in summary_lower or 'hybrid' in job_location_lower:
            parsed_location_type = 'Hybrid'
            
        # --- Parse Salary Data ---
        salary_data = job_row.get('base_salary')
        salary_min = None
        salary_max = None
        salary_currency = None
        salary_period = None

        if isinstance(salary_data, dict):
            salary_min = salary_data.get('min_amount')
            salary_max = salary_data.get('max_amount')
            salary_currency = salary_data.get('currency')
            salary_period = salary_data.get('payment_period')

        # --- Build final job record ---
        job_record = {
            'title': title,
            'description': job_row.get('job_summary', 'No Description'),
            'company_id': company_id,
            'location_name': job_row.get('job_location', 'N/A'),
            'location_type': parsed_location_type,
            'employment_type': job_row.get('job_employment_type'),
            'apply_url': apply_url,
            'posted_at': job_row.get('job_posted_date', 'now()'),
            'source': 'BrightData',
            'is_paid': False,
            'is_featured': False,
            'salary_min': salary_min,
            'salary_max': salary_max,
            'salary_currency': salary_currency,
            'salary_period': salary_period
        }
        
        # --- Use upsert for efficiency ---
        data, count = supabase.table('jobs').upsert(
            job_record,
            on_conflict='apply_url' # Your unique constraint name
        ).execute()

        # Check if a new record was actually inserted
        if data and data[0] and len(data[0]) > 1:
             print(f"✅ Successfully inserted: {title}")
        else:
             print(f"Skipping duplicate: {title}")

    except Exception as e:
        print(f"❌ Error upserting job {title}: {e}")

# --- 3. MAIN SCRIPT LOGIC ---
def main():
    """
    Main function to read the log, download data, and populate the DB.
    """
    print("--- Starting Bright Data Log Processor ---")
    
    try:
        df_logs = pd.read_csv(LOG_FILE)
    except FileNotFoundError:
        print(f"ERROR: Log file not found: {LOG_FILE}")
        return
    except Exception as e:
        print(f"ERROR: Could not read log file: {e}")
        return

    # Filter for successful runs with data
    successful_runs = df_logs[
        (df_logs['status'] == 'ready') &  
        (df_logs['dataset_size'] > 0)
    ]

    print(f"Found {len(successful_runs)} successful snapshots to process.")
    
    # Process one by one
    for snapshot_id in successful_runs['id']:
        print(f"\n--- Processing snapshot: {snapshot_id} ---")
        
        # We don't need params, since the ID is in the URL
        try:
            # Use the URL structure from your script
            url = f"https://api.brightdata.com/datasets/v3/snapshot/{snapshot_id}"
            response = requests.get(
                url,
                headers=BRIGHTDATA_HEADERS,
                timeout=60 # Add a timeout
            )
            response.raise_for_status() # Raise HTTP errors
            
            # --- ⬇️ 2. FIX: PARSE THE NDJSON (NEWLINE-DELIMITED JSON) ---
            print(f"Successfully downloaded {snapshot_id}. Parsing NDJSON...")
            
            raw_text = response.text
            jobs_data = [] # This will be our list of *dictionaries*
            
            # STAGE 1: Parse all lines into a list of dictionaries
            for line in raw_text.splitlines():
                if not line.strip(): # Skip empty lines
                    continue
                try:
                    job_object = json.loads(line) # Parse the *string* line into a *dict*
                    jobs_data.append(job_object)   # Add the *dict* to our list
                except json.JSONDecodeError as e:
                    print(f"Skipping malformed line: {e}")
            
            print(f"Successfully parsed {len(jobs_data)} JSON objects.")
            # --- END OF FIX ---

        except requests.exceptions.RequestException as e:
            print(f"Failed to download {snapshot_id}: {e}")
            continue 
        
        # --- STAGE 2: PROCESS THE PARSED DATA ---
        print(f"Processing {len(jobs_data)} job records...")
        
        for job_row in jobs_data:
            # Now this check will work correctly
            if not isinstance(job_row, dict):
                print(f"Skipping record: not a dictionary. Data: {job_row}")
                continue
            
            # This check will correctly filter out error objects
            if job_row.get('error'):
                print(f"Skipping record: it's an error log ({job_row.get('error_code')})")
                continue

            company_name = job_row.get('company_name')
            apply_url = job_row.get('url')
            print(company_name, apply_url)
            if not company_name or not apply_url:
                print("Skipping job: missing company name or URL.")
                continue

            # 3. GET/CREATE COMPANY
            company_id = get_or_create_company(
                company_name,
                job_row.get('company_url'),
                job_row.get('company_logo')
            )
            
            if not company_id:
                print(f"Skipping job, could not create company: {company_name}")
                continue
            
            # 4. INSERT JOB
            insert_job(job_row, company_id)
            time.sleep(0.05) # Be polite to Supabase API
            
        print(f"Finished processing snapshot {snapshot_id}.")

    print("\n--- All snapshots processed. ---")



In [38]:
main()

--- Starting Bright Data Log Processor ---
Found 35 successful snapshots to process.

--- Processing snapshot: sd_mi1sz8gw2emnoznnzx ---
Successfully downloaded sd_mi1sz8gw2emnoznnzx. Parsing NDJSON...
Successfully parsed 101 JSON objects.
Processing 101 job records...
Skipping record: it's an error log (aborted_page)
TikTok https://www.linkedin.com/jobs/view/machine-learning-engineer-graduate-ads-core-global-2025-start-bs-ms-at-tiktok-4076508901?_l=en
{'name': 'TikTok', 'website_url': 'https://www.linkedin.com/company/tiktok?trk=public_jobs_topcard-org-name', 'logo_url': 'https://media.licdn.com/dms/image/v2/D560BAQHYzDOZhwEZ7Q/company-logo_100_100/B56Zl_CaXSIAAU-/0/1758772964879/tiktok_logo?e=2147483647&v=beta&t=pOXL9bnyBOPMlI34LzNiPIxaX9lk7ry0r40olePnSGE'}
✅ Successfully inserted: Machine Learning Engineer Graduate (Ads Core Global) - 2025 Start (BS/MS)
Take2 Consulting, LLC https://www.linkedin.com/jobs/view/artificial-intelligence-engineer-at-take2-consulting-llc-4335925609?_l=en


In [39]:
import os
import pandas as pd
from supabase import create_client, Client
from dotenv import load_dotenv

# Load environment variables from your .env file
load_dotenv()

# Get your Supabase credentials
url = os.environ.get("SUPABASE_URL")
key = os.environ.get("SUPABASE_KEY")

# Check if credentials are loaded
if not url or not key:
    print("Error: SUPABASE_URL or SUPABASE_KEY not found in .env file.")
else:
    try:
        # 1. Initialize the Supabase client
        supabase: Client = create_client(url, key)
        
        print("Successfully connected to Supabase.")

        # 2. Query the 'jobs' table
        # .select('*') fetches all columns
        response = supabase.table('jobs').select('*').execute()
        
        # 3. Load the data into a pandas DataFrame
        # The actual data is in the 'data' attribute of the response
        df = pd.DataFrame(response.data)

        # 4. Print the DataFrame
        print("Successfully fetched data:")
        print(df)

    except Exception as e:
        print(f"An error occurred: {e}")

Successfully connected to Supabase.
Successfully fetched data:
                                       id                        created_at  \
0    34a3fefd-5ab9-4c43-8d9f-461a3880a099   2025-11-16T16:12:08.78556+00:00   
1    83cec370-e3a9-4ccf-b414-f2a56c7d2f34  2025-11-16T16:12:08.976683+00:00   
2    5a3cb586-e428-46e7-8212-6207bba68520  2025-11-16T16:23:23.969143+00:00   
3    cab5b7f3-b9ac-4218-ab19-b7007da60ecf   2025-11-16T16:12:09.33026+00:00   
4    5b9964ac-8fec-43a9-a5be-82c7341fbadc  2025-11-16T16:08:51.804495+00:00   
..                                    ...                               ...   
995  38e5fa43-5073-4a7a-9f2d-1c85fb3eb0fc  2025-11-16T16:15:04.666118+00:00   
996  d70278d8-1434-4676-931c-59d4865b8c4e  2025-11-16T16:22:36.713517+00:00   
997  ee509342-ae79-4fbe-957a-b0ec5652102e  2025-11-16T16:22:36.925887+00:00   
998  28dfa641-e8b4-49f9-831c-2a700cdcd13b  2025-11-16T16:22:37.132213+00:00   
999  5dddecb3-d22d-47b9-9978-3cee017b4999  2025-11-16T16:22:37.34144

In [41]:
df['location_name'].unique()

array(['Redwood City, CA', 'Sunnyvale, CA', 'Austin, TX',
       'Menlo Park, CA', 'Palo Alto, CA', 'San Francisco Bay Area',
       'San Jose, CA', 'Santa Clara, CA', 'Foster City, CA',
       'Mountain View, CA', 'Cupertino, CA', 'Fremont, CA',
       'Silicon Valley, CA', 'San Mateo, CA', 'Milpitas, CA',
       'Toronto, Ontario, Canada', 'Halifax, Nova Scotia, Canada',
       'Victoria, Nova Scotia, Canada', 'Markham, Ontario, Canada',
       'Canada', 'Vancouver, British Columbia, Canada',
       'Saskatoon, Saskatchewan, Canada', 'Etobicoke, Ontario, Canada',
       'Waterloo, Ontario, Canada', 'Dublin, CA',
       'Montreal, Quebec, Canada', 'Pleasanton, CA',
       'Greater Toronto Area, Canada', 'Ontario, Canada',
       'Gatineau, Quebec, Canada', 'Hamilton, Ontario, Canada',
       'Calgary, Alberta, Canada', 'Mississauga, Ontario, Canada',
       'Brantford, Ontario, Canada', 'Greater Ottawa Metropolitan Area',
       'Ottawa, Ontario, Canada', 'London, Canada Metropolitan 

In [44]:
# --- 3. FETCH DATA AND PROCESS IT ---

try:
    print("Fetching all jobs from database (handling pagination)...")
    
    all_jobs = []
    page_size = 1000
    page = 0
    
    while True:
        start_index = page * page_size
        end_index = start_index + page_size - 1
        
        # Fetch a "page" of data using .range()
        # We select only the columns we need: id and location_name
        response = supabase.table('jobs').select('id, location_name').range(start_index, end_index).execute()
        
        data = response.data
        all_jobs.extend(data)
        
        print(f"Fetched {len(data)} records (page {page + 1})...")
        
        # If we fetched fewer records than the page size, 
        # it means we've reached the end.
        if len(data) < page_size:
            break
            
        page += 1
        # Add a small delay to be kind to the API
        # time.sleep(0.1) 

    df = pd.DataFrame(all_jobs)

    if df.empty:
        print("No job data found.")
    else:
        print(f"\nSuccessfully fetched a total of {len(df)} job records.")
        
        # Apply the parser function to the 'location_name' column
        # and create new columns from the result
        parsed_data = df['location_name'].apply(parse_location)
        df_parsed = pd.json_normalize(parsed_data)
        
        # Combine the new columns with the original DataFrame
        df = df.join(df_parsed)

        # Display the results for verification
        print("\n--- Parsing Results (Sample) ---")
        print(df.head(20)) # Show a sample of the parsed data
        
        # Show all unique new locations for review
        print("\n--- Unique Parsed Locations ---")
        print(df.drop_duplicates(subset=['city', 'state', 'country'])[['location_name', 'city', 'state', 'country']])


        # --- 4. UPDATE THE DATABASE (Run this part when ready) ---
        
        # This section is commented out to prevent accidental writes.
        # Review the output above first. If it looks correct,
        # uncomment the code below and re-run the cell.

        print("\n--- Starting Database Update ---")
        update_count = 0
        
        # Convert DataFrame to a list of dictionaries for the update
        # We only need rows where data actually changed or is new
        updates_to_send = df.where(pd.notnull(df), None).to_dict('records')

        for row in updates_to_send:
            try:
                # Create the payload to send
                update_payload = {
                    'city': row['city'],
                    'state': row['state'],
                    'country': row['country']
                }
                
                # Perform the update
                supabase.table('jobs').update(update_payload).eq('id', row['id']).execute()
                
                update_count += 1
                if update_count % 50 == 0: # Print progress
                    print(f"Updated {update_count} records...")

            except Exception as e:
                print(f"Error updating job {row['id']}: {e}")

        print(f"\n--- Update Complete: {update_count} job records updated. ---")

except Exception as e:
    print(f"An error occurred: {e}")

Fetching all jobs from database (handling pagination)...
Fetched 1000 records (page 1)...
Fetched 1000 records (page 2)...
Fetched 862 records (page 3)...

Successfully fetched a total of 2862 job records.

--- Parsing Results (Sample) ---
                                      id           location_name  \
0   34a3fefd-5ab9-4c43-8d9f-461a3880a099        Redwood City, CA   
1   83cec370-e3a9-4ccf-b414-f2a56c7d2f34           Sunnyvale, CA   
2   5a3cb586-e428-46e7-8212-6207bba68520              Austin, TX   
3   cab5b7f3-b9ac-4218-ab19-b7007da60ecf          Menlo Park, CA   
4   5b9964ac-8fec-43a9-a5be-82c7341fbadc          Menlo Park, CA   
5   da5141a3-4434-455f-8746-5f92da568397           Palo Alto, CA   
6   f1aca45c-9f03-46ec-aa4d-3820dd5822bd  San Francisco Bay Area   
7   8554b51f-67fd-41db-b6e6-b36d684c8090            San Jose, CA   
8   1401ff23-d84c-427d-ba4b-5a048ab6a929         Santa Clara, CA   
9   0d8829c3-724d-40a6-9c4e-a8cc35e6f450            San Jose, CA   
10  ca55637d

In [1]:
# --- SETUP (Ensure your supabase client is initialized as before) ---
import os
from supabase import create_client, Client
url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)

try:
    print("--- Starting Correction of State Names ---")

    # 1. Correct California (CA -> California)
    print("Converting 'CA' to 'California'...")
    response_ca = supabase.table('jobs') \
        .update({'state': 'California'}) \
        .eq('state', 'CA') \
        .execute()
    
    # Count how many were fixed
    count_ca = len(response_ca.data) if response_ca.data else 0
    print(f"✅ Successfully updated {count_ca} records from 'CA' to 'California'.")

    # 2. Correct Texas (TX -> Texas)
    print("Converting 'TX' to 'Texas'...")
    response_tx = supabase.table('jobs') \
        .update({'state': 'Texas'}) \
        .eq('state', 'TX') \
        .execute()

    # Count how many were fixed
    count_tx = len(response_tx.data) if response_tx.data else 0
    print(f"✅ Successfully updated {count_tx} records from 'TX' to 'Texas'.")

    print("\n--- Correction Complete ---")
    print(f"Total records updated: {count_ca + count_tx}")

except Exception as e:
    print(f"An error occurred during the update: {e}")

--- Starting Correction of State Names ---
Converting 'CA' to 'California'...
✅ Successfully updated 1427 records from 'CA' to 'California'.
Converting 'TX' to 'Texas'...
✅ Successfully updated 515 records from 'TX' to 'Texas'.

--- Correction Complete ---
Total records updated: 1942
